In [24]:
import pandas as pd
from langchain import LLMChain, Cohere
from langchain.agents import create_csv_agent
from langchain.llms import Cohere
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import create_pandas_dataframe_agent

In [ ]:
COHERE_API_KEY = "CggzsdnWH6QXtnGJvKYe4IRZyGZ8UkTSykpmAigW"
SERPAPI_API_KEY = "dbc53dd88c7b0957548a81fa162e2d547e03cc19267162a9166e52d4e882f361"

In [ ]:
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
def chat_with_categorized_data(prompt: str):
    agent = create_csv_agent(
        Cohere(model="command-xlarge-nightly", cohere_api_key=COHERE_API_KEY),
        "./unique_category.csv",
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        memory=memory,
    )

    response = agent.run(
        input=prompt
        + "If the answer is generic then return: general else return the closest match to the user interaction. I need only the closest match of the category or general as the output and nothing else..."
    )
    return response

In [ ]:
def filter_dataframe_by_category(data, input_category):
    if isinstance(input_category, str):
        # If input is a string, filter by that single category
        filtered_df = data[data["category"] == input_category]
    elif isinstance(input_category, list):
        # If input is a list, filter by multiple categories
        filtered_df = data[data["category"].isin(input_category)]
    else:
        raise ValueError("Input must be a string or a list")

    return filtered_df

In [ ]:
def chat_with_filtered_data(filtered_df: pd.DataFrame, category: str):
    # agent = create_pandas_dataframe_agent(
    #     ChatOpenAI(
    #         temperature=0, model="gpt-3.5-turbo-0613", openai_api_key=OPENAI_API_KEY
    #     ),
    #     filtered_df,
    #     verbose=True,
    #     agent_type=AgentType.OPENAI_FUNCTIONS,
    # )

    co = Cohere(auth_token=COHERE_API_KEY)
    agent = create_pandas_dataframe_agent(
        llm=LLMChain(llm=co),
        dataframe=filtered_df,
        agent_type="cohere-functions",
        verbose=True,
    )

    response = agent.run(
        "Give a convincing answer to the user in such a way assuming that this is the latest trend as well as of the future."
    )
    return response

In [ ]:
def get_chat_output():
    prompt = "What are the latest trends of 2023 for women in tech..."
    if len(prompt) < 0:
        return {"response": "Please provide a valid prompt."}

    data = pd.read_csv("./data.csv")
    data.drop(["asin", "id"], axis=1, inplace=True)
    data["category"] = data["purl"].apply(lambda x: x.split("/")[-5])

    ans = chat_with_categorized_data(prompt)

    filtered_df = filter_dataframe_by_category(data, ans)
    filtered_df.drop(["img", "purl"], axis=1, inplace=True)

    final_response = chat_with_filtered_data(filtered_df, ans)

    return final_response

In [29]:
prompt = "What are the latest fashion trends of 2023 for women in tech..."
llm = Cohere(
    cohere_api_key=COHERE_API_KEY, model="command-xlarge-nightly", max_tokens=3000
)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)
agent_chain.run(input=prompt)



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: ` The latest fashion trends for women in tech in 2023 are:

1. Athleisure: A fusion of athletic and leisure clothing, athleisure is a comfortable and stylish option for women in tech. Look for clothing that is designed for both comfort and style, such as leggings, hoodies, and sneakers.

2. Minimalist Fashion: A minimalist approach to fashion, this trend focuses on clean lines, neutral colors, and a timeless style. Look for clothing that is simple and classic, with a focus on quality and versatility.

3. Monochromatic Outfits: A sleek and professional look, monochromatic outfits are a great way to show off your style while also being functional and comfortable. Choose a single color and mix and match different shades and textures to create a cohesive and stylish look.

4. Tech-Inspired Fashion: Embrace your tech-inspired fashion with clothing that features unique and innovative designs inspired by technology. Look for clothing that inc